## run this cell before starting spark cluster connection (corrects module error)

In [1]:
# spark.jars.packages com.databricks:spark-avro_2.11:4.0.0
import os
print os.environ['PYSPARK_PYTHON']
os.environ['PYSPARK_PYTHON']=os.environ['LCG_VIEW'] + '/bin/python'
print os.environ['PYSPARK_PYTHON']

/etc/spark/python
/cvmfs/sft.cern.ch/lcg/views/LCG_93/x86_64-slc6-gcc62-opt/bin/python


### Import necessary spark and python stuff

In [2]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.functions import from_json
from pyspark.ml.feature import VectorAssembler

In [3]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', -1)

In [4]:
df_lookup = spark.read.json("/user/sprenner/lookup_final_mai_count.json")

In [5]:
df_lookup.count()

15044

In [6]:
df_mai = spark.read.json("/user/sprenner/training_official_mai_hourly_cleaned_datasets.json")

In [7]:
df_mai = df_mai.withColumn("join_key_mai", concat_ws(':', df_mai["datasetScope"], df_mai["dataset"]))
df_mai.createOrReplaceTempView("mai")

In [8]:
df_joined = df_mai.join(df_lookup, df_mai.join_key_mai == df_lookup.join_key_lookup, 'left_outer')

## add column hour

In [9]:
#column hour Already included from pipeline before!
#df_joined = df_joined.withColumn("hour", from_unixtime(df_joined['traceTimeentryUnix'], 'yyyy-MM-dd HH'))

## check

In [10]:
df_joined.createOrReplaceTempView("joined")

## create index column to later join the partition table correctly

In [11]:
df_joined_indexed = df_joined.sort("traceTimeentryUnix", ascending=True)
df_joined_indexed.createOrReplaceTempView("joined_indexed")
df_joined_indexed = spark.sql("select ROW_NUMBER() OVER (ORDER BY traceTimeentryUnix) as id, * FROM joined_indexed")

In [12]:
df_joined_indexed.createOrReplaceTempView("joined_indexed")

## partition by 2 columns to count amount of files already downloaded in every dataset (uuid alone not enough, collisions).
## only count successful downloads to avoid >100% (STILL POSSIBLE if user downloads data twice)

In [13]:
#https://www.sqlservercentral.com/Forums/573706/ROWNUMBER-and-possibly-group-by
#https://stackoverflow.com/questions/6127338/sql-mysql-select-distinct-unique-but-return-all-columns
df_proportion = spark.sql("SELECT ROW_NUMBER() OVER (PARTITION BY hour, uuid, join_key_mai ORDER BY traceTimeentryUnix ASC) AS Seq , *\
                           FROM (SELECT * FROM \
                                    (SELECT *, ROW_NUMBER() OVER(PARTITION BY hour, uuid, join_key_mai, filename ORDER BY traceTimeentryUnix ASC) AS row_number\
                                     FROM joined_indexed WHERE clientState == 'DONE' AND join_key_lookup IS NOT NULL)\
                                 WHERE row_number == 1) \
                           ORDER BY traceTimeentryUnix ASC")
df_proportion.limit(50).toPandas()

Seq  id   account  accountHash clientState  clientStateHash  \
0   1    2   hegormse -2036406190   DONE       -878527175         
1   2    4   hegormse -2036406190   DONE       -878527175         
2   1    5   toyamaza  368790896    DONE       -878527175         
3   1    6   ysano     1418469109   DONE       -878527175         
4   2    7   toyamaza  368790896    DONE       -878527175         
5   3    8   toyamaza  368790896    DONE       -878527175         
6   1    9   ysano     1418469109   DONE       -878527175         
7   2    10  ysano     1418469109   DONE       -878527175         
8   4    11  toyamaza  368790896    DONE       -878527175         
9   1    12  cylin    -1265559796   DONE       -878527175         
10  1    13  ysano     1418469109   DONE       -878527175         
11  3    14  ysano     1418469109   DONE       -878527175         
12  5    15  toyamaza  368790896    DONE       -878527175         
13  4    16  ysano     1418469109   DONE       -878527175         
14  1    17  ysano     1418469109   DONE       -878527175         
15  1    18  hegormse -2036406190   DONE       -878527175         
16  2    19  hegormse -2036406190   DONE       -878527175         
17  3    20  hegormse -2036406190   DONE       -878527175         
18  4    21  hegormse -2036406190   DONE       -878527175         
19  1    22  cylin    -1265559796   DONE       -878527175         
20  6    23  toyamaza  368790896    DONE       -878527175         
21  1    24  khamano  -1185418172   DONE       -878527175         
22  7    26  toyamaza  368790896    DONE       -878527175         
23  5    27  ysano     1418469109   DONE       -878527175         
24  2    28  ysano     1418469109   DONE       -878527175         
25  6    29  ysano     1418469109   DONE       -878527175         
26  1    30  ysano     1418469109   DONE       -878527175         
27  8    31  toyamaza  368790896    DONE       -878527175         
28  7    32  ysano     1418469109   DONE       -878527175         
29  3    33  ysano     1418469109   DONE       -878527175         
30  1    34  ysano     1418469109   DONE       -878527175         
31  9    36  toyamaza  368790896    DONE       -878527175         
32  8    37  ysano     1418469109   DONE       -878527175         
33  10   38  toyamaza  368790896    DONE       -878527175         
34  1    39  ysano     1418469109   DONE       -878527175         
35  2    40  khamano  -1185418172   DONE       -878527175         
36  11   43  toyamaza  368790896    DONE       -878527175         
37  9    44  ysano     1418469109   DONE       -878527175         
38  12   45  toyamaza  368790896    DONE       -878527175         
39  4    46  ysano     1418469109   DONE       -878527175         
40  2    47  ysano     1418469109   DONE       -878527175         
41  10   48  ysano     1418469109   DONE       -878527175         
42  2    49  ysano     1418469109   DONE       -878527175         
43  11   50  ysano     1418469109   DONE       -878527175         
44  13   51  toyamaza  368790896    DONE       -878527175         
45  12   52  ysano     1418469109   DONE       -878527175         
46  13   53  ysano     1418469109   DONE       -878527175         
47  5    54  ysano     1418469109   DONE       -878527175         
48  3    56  ysano     1418469109   DONE       -878527175         
49  14   57  toyamaza  368790896    DONE       -878527175         

                                                                                                                                          dataset  \
0   mc16_13TeV.364115.Sherpa_221_NNPDF30NNLO_Zee_MAXHTPTV0_70_CFilterBVeto.deriv.NTUP_PILEUP.e5299_e5984_s3126_r10201_r10210_p3384_tid12913234_00   
1   mc16_13TeV.364115.Sherpa_221_NNPDF30NNLO_Zee_MAXHTPTV0_70_CFilterBVeto.deriv.NTUP_PILEUP.e5299_e5984_s3126_r10201_r10210_p3384_tid12913234_00   
2   mc16_13TeV.364169.Sherpa_221_NNPDF30NNLO_Wmunu_MAXHTPTV1000_E_CMS.deriv.DAOD_SUSY5.e5340_e5984_s3126_s3136_r10201_r10210_p3401_tid13555

In [14]:
df_proportion.count()
#ohne join_key_lookup IS NOT NULL: 579934, mit der beschränkung: 578390
# mit : 578390
# werden mit hour evtl mehr weil weniger werte als duplikate gezählt werden?: 579384

357853

In [15]:
df_proportion_percent = df_proportion.withColumn("df_percentage", df_proportion['Seq']/df_proportion['count'])
#df_proportion_percent.limit(50).toPandas()

In [16]:
df_proportion_percent.createOrReplaceTempView("percent")
spark.sql("SELECT * FROM percent WHERE join_key_lookup IS NULL").toPandas()
#sollte nichts gefunden werden

Empty DataFrame
Columns: [Seq, id, account, accountHash, clientState, clientStateHash, dataset, datasetHash, datasetScope, datasetScopeHash, dataset_version, dataset_versionHash, datatype, datatypeHash, eventType, eventTypeHash, eventVersion, eventVersionHash, fileScope, filename, filenameHash, filesize_double, hits, hostname, hostnameHash, hour, hourHash, prod_step, prod_stepHash, project, projectHash, protocol, protocolHash, remoteSite, remoteSiteHash, run_number, run_numberHash, scopeHash, stream_name, stream_nameHash, traceId, traceIdHash, traceIp, traceIpHash, traceTimeentryUnix, transferStart, uuid, uuidHash, join_key_mai, count, join_key_lookup, row_number, df_percentage]
Index: []

In [17]:
#spark.sql("SELECT * FROM percent WHERE id == 893353199188").toPandas()

In [18]:
df_proportion_percent = spark.sql("SELECT CAST(CEILING(df_percentage*100) AS int) AS int_percentage,* FROM percent")

In [19]:
df_proportion_percent.createOrReplaceTempView("percent")

In [20]:
#spark.sql("SELECT * FROM percent WHERE id == 893353199188").toPandas()

In [21]:
spark.sql("SELECT * FROM percent WHERE join_key_lookup IS NULL").toPandas()

Empty DataFrame
Columns: [int_percentage, Seq, id, account, accountHash, clientState, clientStateHash, dataset, datasetHash, datasetScope, datasetScopeHash, dataset_version, dataset_versionHash, datatype, datatypeHash, eventType, eventTypeHash, eventVersion, eventVersionHash, fileScope, filename, filenameHash, filesize_double, hits, hostname, hostnameHash, hour, hourHash, prod_step, prod_stepHash, project, projectHash, protocol, protocolHash, remoteSite, remoteSiteHash, run_number, run_numberHash, scopeHash, stream_name, stream_nameHash, traceId, traceIdHash, traceIp, traceIpHash, traceTimeentryUnix, transferStart, uuid, uuidHash, join_key_mai, count, join_key_lookup, row_number, df_percentage]
Index: []

In [22]:
spark.sql("SELECT COUNT(*) FROM percent WHERE CEILING(df_percentage*100) == 60 AND count > 1").limit(10).toPandas()
#6347
#hourly: 4020?

count(1)
0  2269

## add maximum reached percentage to each uuid

In [23]:
# sind auch 1%er dabei
spark.sql("SELECT uuid, MAX(int_percentage) FROM percent WHERE count > 1 GROUP BY uuid ORDER BY 2 DESC").limit(250).toPandas()

uuid  max(int_percentage)
0    ee96166260064cec9835ade34b425968  100                
1    c02fd86934ca435f80f8528de0a996e3  100                
2    881031969f4548d7bc5ca5e8627c01ea  100                
3    54926183d9cf410fa2d61b270546a8cd  100                
4    7c1b01a1a9fd45d2a622743b8ae14576  100                
5    237ccc940e0c4e41a806002934db94a9  100                
6    f551f39754514d82bcb2c88b22e07a45  100                
7    c74a046df95d456fa23fb2598f2deeba  100                
8    0e174841b085492e960cdda7c35d4b37  100                
9    da97dfe32a9d486ca775b0eb2f506e57  100                
10   21a3ea1a7d14466cb29b93858ecf38ee  100                
11   492b71787d9d411bb1b58731bc3d6a70  100                
12   1b9e0db747754b35849f25fbeb0dcaa7  100                
13   9123b71803494c518d4d7f9ed8b7430d  100                
14   17ae884cc47e4299974f0d985749d5eb  100                
15   f2df8edc6ec3424db8fc34091b02269e  100                
16   b5a0af4705e54b3e9ee9e7142c3bcd79  100                
17   1a5d9b31f2c646e786d9feb646e2a283  100                
18   b8c6bbd7d7a24e2b86f6281e6e94d726  100                
19   ee5ce46974b94f208ed21ebcb55e7ba1  100                
20   d3cb15743e7146d6974d0bda3363feb9  100                
21   9071757def654bebbd7c971de675380b  100                
22   d326df4fcee54e81bd3276f1ab3840a7  100                
23   1a6b575ed04c4d19a085099fe35d066c  100                
24   f37fb88cb2a34f4c96e9b820f4365799  100                
25   9c8dba39160d4e5d91c3c5b4b2f10f29  100                
26   3fafe49c1f444ec39eb0b00596d2c109  100                
27   f0476d3f066440ba965faa02607f6e16  100                
28   7bf59ba05a7f4b9a9c8dbac9cc79a6fb  100                
29   f4d127774c0845dea87a88fc88daf9a1  100                
..                                ...  ...                
220  ef7bcdacdc3144a18a0ed22202910245  100                
221  7398d95f240c430f97b409d39258c4f4  100                
222  ba914e25e50e41399e03620f4187a53c  100                
223  30f6bcec9e0f4d75b9f8dd673a364864  100                
224  921628d3990b4bbf90eddebd977a0ea7  100                
225  72fa74a97cc04600a12a6654d1ef2838  100                
226  8b93144f16c2424481cc51d105940796  100                
227  13ad4ae6b1ef470ea1dde33abfee57a6  100                
228  3037be41aba84cebb8611426599a86ae  100                
229  d67a52fe27df499698f197d6e7502d4e  100                
230  450153fafb8144f9b33e8cd8bb3b102d  100                
231  411b0bb910aa422bb52cbdc45734ce0e  100                
232  9d24231f68f0490eb0150cb343f68d35  100                
233  6bfea26acc4c4af3a3c944a120f1a589  100                
234  24edd311e1d642b7a999d646e0a06bbe  100                
235  032aad2d69bb4f52a94b880796c6be25  100                
236  bf643eb7228a47158c92a10f920cbacf  100                
237  b4ff0fd8885747feae8d612f5f3202b6  100                
238  846721bf854d4319ad6e9cf52d6caf9d  100                
239  74ebf2a98dbe4b92ae51feb07a1282f8  100                
240  04d9b94692d34ef5a2e8d7da03b67d2e  100                
241  6d792f40ec2142c3b31386fecdf6a760  100                
242  c5345a6ba0a54aafb247fa07d46ab8b7  100                
243  168381e7310b4bc594a2a17ed27a4ebe  100                
244  2c149acab4b8450a8c9524b5b8df8753  100                
245  5c18f4a433dd4f0b92492b976638a6f8  100                
246  d6e4c0e6ba584ad08d5b046f00d2a49d  100                
247  d2b98ca94e794d37856c09799984e94c  100                
248  0fd2362add934714a516c683eb531c94  100                
249  8aa4e14446c14935bc0b394e2d424fad  100                

[250 rows x 2 columns]

In [24]:
# sind auch 1%er dabei
spark.sql("SELECT max, COUNT(*) FROM (SELECT uuid, MAX(int_percentage) AS max FROM percent WHERE count > 1 GROUP BY uuid ORDER BY 2 DESC) GROUP BY max").limit(250).toPandas()

max  count(1)
0   100  1772    
1   99   1       
2   98   5       
3   97   11      
4   96   6       
5   95   11      
6   94   14      
7   93   6       
8   92   9       
9   91   4       
10  90   10      
11  89   9       
12  88   12      
13  87   7       
14  86   7       
15  85   7       
16  84   6       
17  83   4       
18  82   4       
19  81   4       
20  80   29      
21  79   3       
22  78   4       
23  77   5       
24  76   4       
25  75   17      
26  74   8       
27  73   8       
28  72   8       
29  71   4       
..  ..  ..       
70  30   9       
71  29   9       
72  28   9       
73  27   10      
74  26   4       
75  25   29      
76  24   16      
77  23   11      
78  22   11      
79  21   3       
80  20   42      
81  19   13      
82  18   10      
83  17   18      
84  16   15      
85  15   27      
86  14   5       
87  13   21      
88  12   20      
89  11   14      
90  10   43      
91  9    26      
92  8    29      
93  7    42      
94  6    31      
95  5    59      
96  4    73      
97  3    94      
98  2    172     
99  1    816     

[100 rows x 2 columns]

In [25]:
spark.sql("SELECT * FROM percent WHERE uuid == '72c49c0c0e1a459a86dc85e45fe546d7' AND count > 1 AND dataset=='mc16_13TeV.363632.MGPy8EG_N30NLO_Wmunu_Ht140_280_BFilter.merge.NTUP_PILEUP.e4944_e5984_s3126_r9364_r9315_p3288_p3126_tid12826358_00' ORDER BY traceTimeentryUnix ASC").toPandas()

Empty DataFrame
Columns: [int_percentage, Seq, id, account, accountHash, clientState, clientStateHash, dataset, datasetHash, datasetScope, datasetScopeHash, dataset_version, dataset_versionHash, datatype, datatypeHash, eventType, eventTypeHash, eventVersion, eventVersionHash, fileScope, filename, filenameHash, filesize_double, hits, hostname, hostnameHash, hour, hourHash, prod_step, prod_stepHash, project, projectHash, protocol, protocolHash, remoteSite, remoteSiteHash, run_number, run_numberHash, scopeHash, stream_name, stream_nameHash, traceId, traceIdHash, traceIp, traceIpHash, traceTimeentryUnix, transferStart, uuid, uuidHash, join_key_mai, count, join_key_lookup, row_number, df_percentage]
Index: []

In [26]:
spark.sql("SELECT * FROM percent WHERE uuid == '50288de2738a423f9b650f404e4da890' AND count > 1 ORDER BY int_percentage ASC").toPandas()

Empty DataFrame
Columns: [int_percentage, Seq, id, account, accountHash, clientState, clientStateHash, dataset, datasetHash, datasetScope, datasetScopeHash, dataset_version, dataset_versionHash, datatype, datatypeHash, eventType, eventTypeHash, eventVersion, eventVersionHash, fileScope, filename, filenameHash, filesize_double, hits, hostname, hostnameHash, hour, hourHash, prod_step, prod_stepHash, project, projectHash, protocol, protocolHash, remoteSite, remoteSiteHash, run_number, run_numberHash, scopeHash, stream_name, stream_nameHash, traceId, traceIdHash, traceIp, traceIpHash, traceTimeentryUnix, transferStart, uuid, uuidHash, join_key_mai, count, join_key_lookup, row_number, df_percentage]
Index: []

In [27]:
df_proportion_percent.createOrReplaceTempView("percent")

In [28]:
spark.sql("SELECT * FROM percent WHERE join_key_lookup IS NULL").toPandas()

Empty DataFrame
Columns: [int_percentage, Seq, id, account, accountHash, clientState, clientStateHash, dataset, datasetHash, datasetScope, datasetScopeHash, dataset_version, dataset_versionHash, datatype, datatypeHash, eventType, eventTypeHash, eventVersion, eventVersionHash, fileScope, filename, filenameHash, filesize_double, hits, hostname, hostnameHash, hour, hourHash, prod_step, prod_stepHash, project, projectHash, protocol, protocolHash, remoteSite, remoteSiteHash, run_number, run_numberHash, scopeHash, stream_name, stream_nameHash, traceId, traceIdHash, traceIp, traceIpHash, traceTimeentryUnix, transferStart, uuid, uuidHash, join_key_mai, count, join_key_lookup, row_number, df_percentage]
Index: []

In [29]:
df_proportion_percent_dropped = df_proportion_percent.selectExpr("id as id_proportion", "int_percentage")
df_proportion_percent_dropped.limit(10).toPandas()

id_proportion  int_percentage
0  2              10            
1  4              20            
2  5              1             
3  6              1             
4  7              1             
5  8              2             
6  9              1             
7  10             1             
8  11             2             
9  12             1

In [30]:
#df_final = df_joined_indexed.join(df_proportion_percent_dropped, df_joined_indexed.id == df_proportion_percent_dropped.id_proportion, "left_outer")
df_joined_indexed.createOrReplaceTempView("joined_indexed")
df_proportion_percent_dropped.createOrReplaceTempView("percent_dropped")
df_final = spark.sql("SELECT * FROM joined_indexed LEFT JOIN percent_dropped ON joined_indexed.id == percent_dropped.id_proportion")

In [31]:
df_final.createOrReplaceTempView("final")
spark.sql("SELECT * FROM final WHERE join_key_lookup IS NULL").toPandas()

id   account  accountHash         clientState  clientStateHash  \
0      51757   aknue     720293188    DONE               -878527175         
1      51767   aknue     720293188    DONE               -878527175         
2      51773   aknue     720293188    DONE               -878527175         
3      51795   aknue     720293188    DONE               -878527175         
4      51805   aknue     720293188    DONE               -878527175         
5      51818   aknue     720293188    DONE               -878527175         
6      51862   aknue     720293188    DONE               -878527175         
7      51894   aknue     720293188    DONE               -878527175         
8      51904   aknue     720293188    DONE               -878527175         
9      51929   aknue     720293188    DONE               -878527175         
10     51955   aknue     720293188    DONE               -878527175         
11     51965   aknue     720293188    DONE               -878527175         
12     51998   aknue     720293188    DONE               -878527175         
13     52030   aknue     720293188    DONE               -878527175         
14     52038   aknue     720293188    DONE               -878527175         
15     52095   aknue     720293188    DONE               -878527175         
16     52099   aknue     720293188    DONE               -878527175         
17     52104   aknue     720293188    DONE               -878527175         
18     52141   aknue     720293188    DONE               -878527175         
19     52143   aknue     720293188    DONE               -878527175         
20     53072   lcorpe   -279559867    DONE               -878527175         
21     53220   cdelport -1042421268   DONE               -878527175         
22     53221   cdelport -1042421268   DONE               -878527175         
23     73811   luambroz -1063736139   DONE               -878527175         
24     73912   luambroz -1063736139   DONE               -878527175         
25     73941   luambroz -1063736139   DONE               -878527175         
26     73968   luambroz -1063736139   DONE               -878527175         
27     74027   luambroz -1063736139   DONE               -878527175         
28     74087   luambroz -1063736139   DONE               -878527175         
29     74127   luambroz -1063736139   DONE               -878527175         
...      ...        ...         ...    ...                      ...         
11121  581620  liuya     700350695    DONE               -878527175         
11122  581668  liuya     700350695    DONE               -878527175         
11123  581670  liuya     700350695    DONE               -878527175         
11124  581671  liuya     700350695    DONE               -878527175         
11125  581977  ahasib    79838733     DONE               -878527175         
11126  582332  vkitali   1625481889   DONE               -878527175         
11127  582334  vkitali   1625481889   DONE               -878527175         
11128  583193  vkitali   1625481889   DONE               -878527175         
11129  583196  vkitali   1625481889   DONE               -878527175         
11130  583210  vkitali   1625481889   DONE               -878527175         
11131  583214  vkitali   1625481889   DONE               -878527175         
11132  583215  vkitali   1625481889   DONE               -878527175         
11133  584843  kmistry  -1590875906   DONE               -878527175         
11134  586539  ereynold  1766974890   ServiceUnavailable -1394468120        
11135  586540  ereynold  1766974890   ServiceUnavailable -1394468120        
11136  586542  ereynold  1766974890   ServiceUnavailable -1394468120        
11137  586559  ereynold  1766974890   ServiceUnavailable -1394468120        
11138  586560  ereynold  1766974890   ServiceUnavailable -1394468120        
11139  586561  ereynold  1766974890   ServiceUnavailable -1394468120        
11140  587642  ahasib    79838733     DONE               -878527175         
11141  

In [32]:
df_final.count()

589283

In [33]:
df_final.sort("traceTimeentryUnix", ascending=True).limit(10).toPandas()

id   account  accountHash         clientState  clientStateHash  \
0  1   hegormse -2036406190   ServiceUnavailable -1394468120        
1  2   hegormse -2036406190   DONE               -878527175         
2  3   hegormse -2036406190   ServiceUnavailable -1394468120        
3  4   hegormse -2036406190   DONE               -878527175         
4  5   toyamaza  368790896    DONE               -878527175         
5  6   ysano     1418469109   DONE               -878527175         
6  7   toyamaza  368790896    DONE               -878527175         
7  8   toyamaza  368790896    DONE               -878527175         
8  9   ysano     1418469109   DONE               -878527175         
9  10  ysano     1418469109   DONE               -878527175         

                                                                                                                                         dataset  \
0  mc16_13TeV.364115.Sherpa_221_NNPDF30NNLO_Zee_MAXHTPTV0_70_CFilterBVeto.deriv.NTUP_PILEUP.e5299_e5984_s3126_r10201_r10210_p3384_tid12913234_00   
1  mc16_13TeV.364115.Sherpa_221_NNPDF30NNLO_Zee_MAXHTPTV0_70_CFilterBVeto.deriv.NTUP_PILEUP.e5299_e5984_s3126_r10201_r10210_p3384_tid12913234_00   
2  mc16_13TeV.364115.Sherpa_221_NNPDF30NNLO_Zee_MAXHTPTV0_70_CFilterBVeto.deriv.NTUP_PILEUP.e5299_e5984_s3126_r10201_r10210_p3384_tid12913234_00   
3  mc16_13TeV.364115.Sherpa_221_NNPDF30NNLO_Zee_MAXHTPTV0_70_CFilterBVeto.deriv.NTUP_PILEUP.e5299_e5984_s3126_r10201_r10210_p3384_tid12913234_00   
4  mc16_13TeV.364169.Sherpa_221_NNPDF30NNLO_Wmunu_MAXHTPTV1000_E_CMS.deriv.DAOD_SUSY5.e5340_e5984_s3126_s3136_r10201_r10210_p3401_tid13555633_00   
5  data17_13TeV.00336832.physics_Main.deriv.DAOD_EXOT8.f868_m1879_p3481_tid13457653_00                                                             
6  mc16_13TeV.364169.Sherpa_221_NNPDF30NNLO_Wmunu_MAXHTPTV1000_E_CMS.deriv.DAOD_SUSY5.e5340_e5984_s3126_s3136_r10201_r10210_p3401_tid13555633_00   
7  mc16_13TeV.364169.Sherpa_221_NNPDF30NNLO_Wmunu_MAXHTPTV1000_E_CMS.deriv.DAOD_SUSY5.e5340_e5984_s3126_s3136_r10201_r10210_p3401_tid13555633_00   
8  data17_13TeV.00331085.physics_Main.deriv.DAOD_EXOT8.f848_m1844_p3481_tid13457361_00                                                             
9  data17_13TeV.00336832.physics_Main.deriv.DAOD_EXOT8.f868_m1879_p3481_tid13457653_00                                                             

   datasetHash  datasetScope  datasetScopeHash  \
0 -1708451030   mc16_13TeV   -1447489496         
1 -1708451030   mc16_13TeV   -1447489496         
2 -1708451030   mc16_13TeV   -1447489496         
3 -1708451030   mc16_13TeV   -1447489496         
4  405008816    mc16_13TeV   -1447489496         
5 -141444347    data17_13TeV -841583270          
6  405008816    mc16_13TeV   -1447489496         
7  405008816    mc16_13TeV   -1447489496         
8  781487554    data17_13TeV -841583270          
9 -141444347    data17_13TeV -841583270          

                                              dataset_version  \
0  e5299_e5984_s3126_r10201_r10210_p3384_tid12913234_00         
1  e5299_e5984_s3126_r10201_r10210_p3384_tid12913234_00         
2  e5299_e5984_s3126_r10201_r10210_p3384_tid12913234_00         
3  e5299_e5984_s3126_r10201_r10210_p3384_tid12913234_00         
4  e5340_e5984_s3126_s3136_r10201_r10210_p3401_tid13555633_00   
5  f868_m1879_p3481_tid13457653_00                              
6  e5340_e5984_s3126_s3136_r10201_r10210_p3401_tid13555633_00   
7  e5340_e5984_s3126_s3136_r10201_r10210_p3401_tid13555633_00   
8  f848_m1844_p3481_tid13457361_00                              
9  f868_m1879_p3481_tid13457653_00                              

   dataset_versionHash     datatype  datatypeHash eventType  eventTypeHash  \
0 -1154314823           NTUP_PILEUP  1967069222    download  59378080        
1 -1154314823           NTUP_PILEUP  1967069222    download  59378080        
2 -1154314823           NTUP_PILEUP  1967069222    download  59378080        
3 -1154314823           NTUP_PILEUP

In [34]:
df_final = df_final.sort("traceTimeentryUnix", ascending=True)

In [35]:
#df_final.write.format("json").save("/user/sprenner/training_official_mai_hourly_proportion.json")

## check if hourly works as expected

In [36]:
df_final.createOrReplaceTempView("final_check")
spark.sql("SELECT uuid,(max-min)/3600 AS diff_hours, from_unixtime(min) AS min, from_unixtime(max) AS max \
           FROM(SELECT uuid, MIN(traceTimeentryUnix) AS min, MAX(traceTimeentryunix) AS max\
           FROM final_check WHERE count > 1 GROUP BY uuid)\
           WHERE min != max AND (max-min)/3600 > 1.0\
           ORDER BY diff_hours DESC").toPandas()

uuid  diff_hours                  min  \
0    4601b69722da41a189695ad839482940  265.904798  2018-05-01 00:04:16   
1    388ce711d1e041fca09f118c449414ea  185.209385  2018-05-01 00:05:50   
2    485a4dac22924ac0861f2eb862082336  179.182784  2018-05-01 00:00:50   
3    23c79995674e44a3af7bf767a2f67242  178.797765  2018-05-01 00:07:32   
4    7212d2b31c8141619ee0cac7aaa4c36e  168.694153  2018-05-24 14:54:46   
5    9e95d14a48d74cb499f8a45144f4cbe9  168.104803  2018-05-15 14:13:54   
6    91b055e15d3346cb9bdc657f779f5765  167.953035  2018-05-15 14:22:59   
7    dec883d36aca4beba39074bb19719e43  162.848521  2018-05-01 00:01:51   
8    ad62b363fa4f4380bfa9a07d986eca3e  156.565017  2018-05-15 14:10:36   
9    5ea5d27be0f84a0a8c537dfd9f6c0f27  155.983729  2018-05-15 14:17:09   
10   960095d369c04c6fb7c52b01182c7840  136.556431  2018-05-01 00:02:54   
11   7c80a0e7e13d4191bbc3a0ef593b74e5  125.098498  2018-05-18 22:15:43   
12   1de595af32704092872b3ab9487fc821  102.604839  2018-05-15 14:15:34   
13   6b66e0ff799e42cf9c9f8c83fb9c0c59  95.156734   2018-05-03 11:36:58   
14   7af8ea8637e841f9af624d4542bf2999  93.532513   2018-05-01 00:03:29   
15   6be209edbf274393929b9571f20b6212  88.417655   2018-05-15 14:14:15   
16   d3d37a690ca34272bad7ccd02bca88d5  83.172193   2018-05-04 00:04:54   
17   386c02aa067d48ee94e05653dda8b1e4  79.999470   2018-05-15 14:19:41   
18   2e40d4e287304d888e0fa2c50a112414  74.768914   2018-05-04 13:18:40   
19   b52b801e590643528427a5d0d9f80a3f  73.409006   2018-05-01 00:06:52   
20   a6ca3d40d9144bdfb84bb02f48577e89  68.274443   2018-05-22 14:30:00   
21   1a3848afdc504a86a9f67dd2bc7c81f1  64.006540   2018-05-15 14:10:09   
22   3e65b092b8674a3e8c52e057d4c4c27a  54.206698   2018-05-25 10:14:55   
23   2321f5cc35414d26a67897049c0578e3  51.087601   2018-05-07 11:40:15   
24   4844bd9aed8c4519a87838e0d857e325  46.463838   2018-05-09 15:33:44   
25   6344a0da589a4c81b153e11b26aef517  43.836771   2018-05-25 10:53:16   
26   137315a488f74e58b4a08954e1cf0890  39.852431   2018-05-15 14:08:38   
27   9f52628ba2bd4545bc24a2871fdc6e80  39.740902   2018-05-17 19:35:20   
28   336d302cb0fc49f48d4b6dc670677af6  36.659512   2018-05-13 17:18:23   
29   c1705fad9f5840eabbb822dc955619e2  32.873870   2018-05-15 10:00:25   
..                                ...        ...                   ...   
496  a343a8c36b6d49b4a9a02442090086ae  1.081827    2018-05-07 07:20:54   
497  f0529f5988944bb0baf8447c9566dda3  1.079001    2018-05-22 12:50:16   
498  32d826e22308438c83888abee08812cc  1.074685    2018-05-02 10:15:25   
499  5d1f6a4705f74c54b7dbb53c7974e6a9  1.074233    2018-05-16 15:36:13   
500  c89eca163cf747ff84138a8ff6ebfe92  1.073683    2018-05-26 01:02:31   
501  ceb8f58c65e748e1b794503c0f018a27  1.070654    2018-05-11 04:10:55   
502  f13233c103cc41f897628e19b77c98fd  1.069852    2018-05-29 19:14:26   
503  e45dee5f96ab4637b0c09528fad518bf  1.068928    2018-05-26 10:36:14   
504  2d481879e16349479235f3c2da9fef09  1.067617    2018-05-23 12:30:04   
505  baff140ad6a84218be11ec8aad858699  1.056842    2018-05-15 15:09:55   
506  9708a50169a5484e891dacd20a8199ba  1.055986    2018-05-31 13:09:14   
507  f6dab2ec47f84201ac0f91e3dbb76cd0  1.052605    2018-05-23 19:12:44   
508  63a404dbbe864e62866ebfea5c6fa4df  1.047840    2018-05-17 14:46:15   
509  d1913dcd446f4b348fbac0bd913b8901  1.044717    2018-05-12 09:44:03   
510  3f36cdbddc934283937e142a1abdba31  1.043276    2018-05-23 12:34:23   
511  cef727f45bee46bbba05495fe96f4d1b  1.039656    2018-05-20 03:50:01   
512  4c14a4ffbcaa4e5994d25152e8c1a687  1.037185    2018-05-26 00:25:02   
513  8dd3fe59a6f94112a0142ef205587b43  1.034236    2018-05-17 13:54:27   
514  a50760fffbc34945bc9d81cdc1acf591  1.028620    2018-05-30 16:02:12   
515  7c107f5280fe409cbade1a54ac9911d3  1.028436    2018-05-23 00:47:45   
516  c91befe0beaa44c0adf6619c70ef3b79  1.026280    2018-05-11 14:39:38   
517  d6e686c93e614f0da157a3dcf75eabe1  1.026239    2018-05-15 11:50:45   
518  93b3ed20a617427782bab548482b28e8

In [37]:
spark.sql("SELECT * FROM final_check").limit(100).toPandas()

id   account  accountHash         clientState  clientStateHash  \
0   1    hegormse -2036406190   ServiceUnavailable -1394468120        
1   2    hegormse -2036406190   DONE               -878527175         
2   3    hegormse -2036406190   ServiceUnavailable -1394468120        
3   4    hegormse -2036406190   DONE               -878527175         
4   5    toyamaza  368790896    DONE               -878527175         
5   6    ysano     1418469109   DONE               -878527175         
6   7    toyamaza  368790896    DONE               -878527175         
7   8    toyamaza  368790896    DONE               -878527175         
8   9    ysano     1418469109   DONE               -878527175         
9   10   ysano     1418469109   DONE               -878527175         
10  11   toyamaza  368790896    DONE               -878527175         
11  12   cylin    -1265559796   DONE               -878527175         
12  13   ysano     1418469109   DONE               -878527175         
13  14   ysano     1418469109   DONE               -878527175         
14  15   toyamaza  368790896    DONE               -878527175         
15  16   ysano     1418469109   DONE               -878527175         
16  17   ysano     1418469109   DONE               -878527175         
17  18   hegormse -2036406190   DONE               -878527175         
18  19   hegormse -2036406190   DONE               -878527175         
19  20   hegormse -2036406190   DONE               -878527175         
20  21   hegormse -2036406190   DONE               -878527175         
21  22   cylin    -1265559796   DONE               -878527175         
22  23   toyamaza  368790896    DONE               -878527175         
23  24   khamano  -1185418172   DONE               -878527175         
24  25   khamano  -1185418172   ServiceUnavailable -1394468120        
25  26   toyamaza  368790896    DONE               -878527175         
26  27   ysano     1418469109   DONE               -878527175         
27  28   ysano     1418469109   DONE               -878527175         
28  29   ysano     1418469109   DONE               -878527175         
29  30   ysano     1418469109   DONE               -878527175         
..  ..     ...            ...    ...                      ...         
70  71   ysano     1418469109   DONE               -878527175         
71  72   ysano     1418469109   DONE               -878527175         
72  73   toyamaza  368790896    DONE               -878527175         
73  74   toyamaza  368790896    DONE               -878527175         
74  75   toyamaza  368790896    DONE               -878527175         
75  76   ysano     1418469109   DONE               -878527175         
76  77   toyamaza  368790896    DONE               -878527175         
77  78   toyamaza  368790896    DONE               -878527175         
78  79   ysano     1418469109   DONE               -878527175         
79  80   ysano     1418469109   DONE               -878527175         
80  81   ysano     1418469109   DONE               -878527175         
81  82   ysano     1418469109   DONE               -878527175         
82  83   ysano     1418469109   DONE               -878527175         
83  84   ysano     1418469109   DONE               -878527175         
84  85   ysano     1418469109   DONE               -878527175         
85  86   ysano     1418469109   DONE               -878527175         
86  87   ysano     1418469109   DONE               -878527175         
87  88   ysano     1418469109   ServiceUnavailable -1394468120        
88  89   ysano     1418469109   DONE               -878527175         
89  90   toyamaza  368790896    DONE               -878527175         
90  91   ysano     1418469109   DONE               -878527175         
91  92   toyamaza  368790896    DONE               -878527175         
92  93   acukierm  1777994965   DONE               -878527175         
93  94   ysano     1418469109   DONE               -878527175         
94  95   ysano     1418469109

## create table with maximum percent for each hour/uuid/dataset combination

In [38]:
df_total_percent = spark.sql("SELECT hour as hour_total, uuid as uuid_total, join_key_mai as join_key_mai_total,MAX(int_percentage) as max_percentage FROM final_check GROUP BY hour_total, uuid_total, join_key_mai_total")

## join maximum percentages to the corresponding datasets per hour

In [39]:
df_total_percent.createOrReplaceTempView('total')
df_training_prefetching = spark.sql("SELECT * FROM final_check INNER JOIN total ON final_check.hour == total.hour_total \
                                     AND final_check.uuid == total.uuid_total AND final_check.join_key_mai == total.join_key_mai_total\
                                     SORT BY traceTimeentryUnix ASC")

In [40]:
df_training_prefetching = df_training_prefetching.drop('hour_total','uuid_total','join_key_mai_total')

In [45]:
df_training_prefetching.where("uuid == 'a343a8c36b6d49b4a9a02442090086ae' AND join_key_mai == 'mc16_13TeV:mc16_13TeV.341423.PowhegPythia8EvtGen_CT10_AZNLO_WmH125J_MINLO_muvmuWWlvlv.deriv.DAOD_STDM5.e4210_e5984_s3126_r9364_r9315_p3522_tid13997631_00'").toPandas()

id account  accountHash clientState  clientStateHash  \
0  144739  tuna    593210421    DONE       -878527175         
1  144744  tuna    593210421    DONE       -878527175         
2  144770  tuna    593210421    DONE       -878527175         
3  144951  tuna    593210421    DONE       -878527175         
4  145017  tuna    593210421    DONE       -878527175         
5  145028  tuna    593210421    DONE       -878527175         
6  145143  tuna    593210421    DONE       -878527175         
7  145167  tuna    593210421    DONE       -878527175         

                                                                                                                                          dataset  \
0  mc16_13TeV.341423.PowhegPythia8EvtGen_CT10_AZNLO_WmH125J_MINLO_muvmuWWlvlv.deriv.DAOD_STDM5.e4210_e5984_s3126_r9364_r9315_p3522_tid13997631_00   
1  mc16_13TeV.341423.PowhegPythia8EvtGen_CT10_AZNLO_WmH125J_MINLO_muvmuWWlvlv.deriv.DAOD_STDM5.e4210_e5984_s3126_r9364_r9315_p3522_tid13997631_00   
2  mc16_13TeV.341423.PowhegPythia8EvtGen_CT10_AZNLO_WmH125J_MINLO_muvmuWWlvlv.deriv.DAOD_STDM5.e4210_e5984_s3126_r9364_r9315_p3522_tid13997631_00   
3  mc16_13TeV.341423.PowhegPythia8EvtGen_CT10_AZNLO_WmH125J_MINLO_muvmuWWlvlv.deriv.DAOD_STDM5.e4210_e5984_s3126_r9364_r9315_p3522_tid13997631_00   
4  mc16_13TeV.341423.PowhegPythia8EvtGen_CT10_AZNLO_WmH125J_MINLO_muvmuWWlvlv.deriv.DAOD_STDM5.e4210_e5984_s3126_r9364_r9315_p3522_tid13997631_00   
5  mc16_13TeV.341423.PowhegPythia8EvtGen_CT10_AZNLO_WmH125J_MINLO_muvmuWWlvlv.deriv.DAOD_STDM5.e4210_e5984_s3126_r9364_r9315_p3522_tid13997631_00   
6  mc16_13TeV.341423.PowhegPythia8EvtGen_CT10_AZNLO_WmH125J_MINLO_muvmuWWlvlv.deriv.DAOD_STDM5.e4210_e5984_s3126_r9364_r9315_p3522_tid13997631_00   
7  mc16_13TeV.341423.PowhegPythia8EvtGen_CT10_AZNLO_WmH125J_MINLO_muvmuWWlvlv.deriv.DAOD_STDM5.e4210_e5984_s3126_r9364_r9315_p3522_tid13997631_00   

   datasetHash datasetScope  datasetScopeHash  \
0  1016814249   mc16_13TeV  -1447489496         
1  1016814249   mc16_13TeV  -1447489496         
2  1016814249   mc16_13TeV  -1447489496         
3  1016814249   mc16_13TeV  -1447489496         
4  1016814249   mc16_13TeV  -1447489496         
5  1016814249   mc16_13TeV  -1447489496         
6  1016814249   mc16_13TeV  -1447489496         
7  1016814249   mc16_13TeV  -1447489496         

                                      dataset_version  dataset_versionHash  \
0  e4210_e5984_s3126_r9364_r9315_p3522_tid13997631_00  1041626473            
1  e4210_e5984_s3126_r9364_r9315_p3522_tid13997631_00  1041626473            
2  e4210_e5984_s3126_r9364_r9315_p3522_tid13997631_00  1041626473            
3  e4210_e5984_s3126_r9364_r9315_p3522_tid13997631_00  1041626473            
4  e4210_e5984_s3126_r9364_r9315_p3522_tid13997631_00  1041626473            
5  e4210_e5984_s3126_r9364_r9315_p3522_tid13997631_00  1041626473            
6  e4210_e5984_s3126_r9364_r9315_p3522_tid13997631_00  1041626473            
7  e4210_e5984_s3126_r9364_r9315_p3522_tid13997631_00  1041626473            

     datatype  datatypeHash eventType  eventTypeHash eventVersion  \
0  DAOD_STDM5  1104055723    download  59378080       1.14.11       
1  DAOD_STDM5  1104055723    download  59378080       1.14.11       
2  DAOD_STDM5  1104055723    download  59378080       1.14.11       
3  DAOD_STDM5  1104055723    download  59378080       1.14.11       
4  DAOD_STDM5  1104055723    download  59378080       1.14.11       
5  DAOD_STDM5  1104055723    download  59378080       1.14.11       
6  DAOD_STDM5  1104055723    download  59378080       1.14.11       
7  DAOD_STDM5  1104055723    download  59378080       1.14.11       

   eventVersionHash   fileScope                                 filename  \
0 -2044360925        mc16_13TeV  DAOD_STDM5.13997631._000002.pool.root.1   
1 -2044360925        mc16_13TeV  DAOD_STDM5.13997631._000001.pool.root.1   
2 -2044360925        mc16_13TeV  DAOD_STDM5.13997631._000003.pool.root.1   
3 -2044360925        mc16_

In [46]:
df_training_prefetching = df_training_prefetching.sort("traceTimeentryUnix", ascending=True)

In [47]:
df_training_prefetching.write.format("json").save("/user/sprenner/training_official_mai_hourly_proportion_prefetching.json")